In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

## Womens Shoes

In [13]:
women_link=[]
for i in range(1,10):
    try:
        result=requests.get(f"https://nemital.com/collections/for-her?page={i}",headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        prod=soup.find_all('div',{'class':'grid-view-item product-card'})
        for j in prod:
            women_link.append('https://nemital.com'+j.find('a').attrs['href'])
    except:
        pass 

In [81]:
len(women_link)

59

In [82]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [83]:
for i in tqdm(women_link):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        product_name=soup.find('title').text
        product_brand=soup.find('meta',{'property':"og:site_name"}).attrs['content']
        product_link=i
        product_cat='Footwear'
        product_des=soup.find('meta',{'property':'og:description'}).attrs['content']

        product_mrp=int(float(soup.find('meta',{'property':'og:price:amount'}).attrs['content']))
        product_price=int(float(soup.find('meta',{'property':'og:price:amount'}).attrs['content']))
        img=soup.find_all('meta',{'property':'og:image'})
        images=[]
        for i in img:
            images.append(i.attrs['content'])
        primary_link=images[0]
        if(len(images)>1):
            secondary_link=','.join(images[1:])
        else:
            secondary_link="Not Available"
        size=soup.find('select',{'id':'SingleOptionSelector-0'}).find_all('option')
        sizes=[]
        for i in size:
            sizes.append(i.attrs['value'])
        size_avl=','.join(sizes)
    
    except:
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 59/59 [00:48<00:00,  1.21it/s]


In [84]:
out=pd.DataFrame({
    
    "Website":'https://nemital.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [85]:
out.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://nemital.com/,https://nemital.com/collections/for-her/produc...,Mules - Grey\n– Nemital,Nemital,Footwear,"6,7,8,9,10,11",900,900,Women,Category:WomenDescription: These slip-ons or w...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
1,https://nemital.com/,https://nemital.com/collections/for-her/produc...,Mules - Blue\n– Nemital,Nemital,Footwear,"6,7,8,9,10,11",900,900,Women,Category:WomenDescription: These slip-ons or w...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
2,https://nemital.com/,https://nemital.com/collections/for-her/produc...,Baarish | बारिश - Shimmery Grey\n– Nemital,Nemital,Footwear,"36,37,38,39,40,41",850,850,Women,Sandals with Elastic Introducing 'Baarish' - t...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
3,https://nemital.com/,https://nemital.com/collections/for-her/produc...,Baarish | बारिश - Floral Blue\n– Nemital,Nemital,Footwear,"36,37,38,39,40,41",850,850,Women,Sandals with Elastic Introducing 'Baarish' - t...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
4,https://nemital.com/,https://nemital.com/collections/for-her/produc...,Baarish | बारिश - Brown\n– Nemital,Nemital,Footwear,"36,37,38,39,40,41",850,850,Women,Sandals with Elastic Introducing 'Baarish' - t...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,


In [86]:
conn = sqlite3.connect(r'nemital.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
conn.commit()

out.to_sql('product_details', conn, if_exists='replace', index = False)
conn.close()

## Mens Shoes

In [15]:
mens_link=[]
for i in range(1,3):
    try:
        result=requests.get(f"https://nemital.com/collections/for-him?page={i}",headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        prod=soup.find_all('div',{'class':'grid-view-item product-card'})
        for j in prod:
            mens_link.append('https://nemital.com'+j.find('a').attrs['href'])
    except:
        pass 

In [87]:
len(mens_link)

12

In [88]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [89]:
for i in tqdm(mens_link):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        product_name=soup.find('title').text
        product_brand=soup.find('meta',{'property':"og:site_name"}).attrs['content']
        product_link=i
        product_cat='Footwear'
        product_des=soup.find('meta',{'property':'og:description'}).attrs['content']

        product_mrp=int(float(soup.find('meta',{'property':'og:price:amount'}).attrs['content']))
        product_price=int(float(soup.find('meta',{'property':'og:price:amount'}).attrs['content']))
        img=soup.find_all('meta',{'property':'og:image'})
        images=[]
        for i in img:
            images.append(i.attrs['content'])
        primary_link=images[0]
        if(len(images)>1):
            secondary_link=','.join(images[1:])
        else:
            secondary_link="Not Available"
        size=soup.find('select',{'id':'SingleOptionSelector-0'}).find_all('option')
        sizes=[]
        for i in size:
            sizes.append(i.attrs['value'])
        size_avl=','.join(sizes)
    
    except:
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 12/12 [00:09<00:00,  1.22it/s]


In [90]:
out=pd.DataFrame({
    
    "Website":'https://nemital.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [91]:
out.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://nemital.com/,https://nemital.com/collections/for-him/produc...,Parampara | परंपरा - Yellow\n– Nemital,Nemital,Footwear,"40,41,43,42,44,45",920,920,Men,Festive Kolhapuri (without leather) (If your s...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
1,https://nemital.com/,https://nemital.com/collections/for-him/produc...,Parampara | परंपरा - Silk\n– Nemital,Nemital,Footwear,"40,41,43,42,44,45",920,920,Men,Festive Kolhapuri (without leather) (If your s...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
2,https://nemital.com/,https://nemital.com/collections/for-him/produc...,Shishir | शिषिर - Blue\n– Nemital,Nemital,Footwear,"39,40,41,42,43,44,45",800,800,Men,Cross Belts with cork base It features a jute ...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
3,https://nemital.com/,https://nemital.com/collections/for-him/produc...,Nitya | नित्य - Contrast Black\n– Nemital,Nemital,Footwear,"39,40,41,42,43,44,45",870,870,Men,Mens chappals with toe ring contrast belt It f...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,
4,https://nemital.com/,https://nemital.com/collections/for-him/produc...,Grishma | ग्रीष्म - Grey\n– Nemital,Nemital,Footwear,"39,40,41,42,43,44,45",850,850,Men,Kolhapuri (without leather) in Pintuck lines F...,http://cdn.shopify.com/s/files/1/0450/7611/971...,http://cdn.shopify.com/s/files/1/0450/7611/971...,


In [92]:
conn = sqlite3.connect(r'nemital.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()

out.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()